In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    # "atr",
    "trend",
    # "trend_profit",
    # "trend_multi",
    # "clusters",
    # "multi_window",
    # "validated_levels",
    # "zigzag",
    # "mean_rev",
    # "mean_rev_multi",
    # "mean_rev_vol",
    # "filter",
    # "multi_filter",
    # "filter_bidirectional",
    # "filter_one",
    # "trend_one",
    # "filter_flat",
]

base_cfg = dict(
    symbol='XAUUSD', timeframe='H1', direction='both',
    train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    test_start=datetime(2021,6,1),  test_end=datetime(2022,6,1),
    search_type='lgmm', n_trials=1000, n_models=1
)

configs = []
for lm in label_methods:
    cfg = base_cfg.copy()
    cfg["label_method"] = lm
    configs.append(cfg)

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"] = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['label_method']} {cfg['search_type']} {cfg.get('search_subtype', '')}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD H1 both trend lgmm] modelo 0 trial 1/1000 ins=-1.000000 oos=-1.000000 avg=3.80s mem=436.55MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 2/1000 ins=-1.000000 oos=-1.000000 avg=4.37s mem=846.71MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 3/1000 ins=-1.000000 oos=-1.000000 avg=4.30s mem=1128.96MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 4/1000 ins=0.035944 oos=0.011422 avg=8.97s mem=1303.45MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 5/1000 ins=0.035944 oos=0.011422 avg=7.23s mem=1301.22MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 6/1000 ins=0.035944 oos=0.011422 avg=6.10s mem=1301.22MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 7/1000 ins=0.035944 oos=0.011422 avg=5.59s mem=1301.22MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 8/1000 ins=0.035944 oos=0.011422 avg=4.99s mem=1301.22MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 9/1000 ins=0.035944 oos=0.011422 avg=5.43s mem=1358.52MB 
[XAUUSD H1 both trend lgmm] modelo 0 trial 10/1000 ins=0.035944 oos=0.011422 avg=4.94s 